In [ ]:
#hide
from dash_oop_components.core import *

# Covid Dashboard Example
> A small example showing the power of dash_oop_components in building a modular covid tracking dashboard




This dashboard has been deployed to [https://dash-oop-demo.herokuapp.com/](https://dash-oop-demo.herokuapp.com/)

# Dashboard Design

The dashboard consists one `DashFigureFactory` and four `DashComponent`s. 

1. **CovidPlots**: `DashFigureFactory` that stores a covid dataset and can plot both timeseries and piecharts for a select number of countries for either cases or deaths.
2. **CovidTimeSeries**: a component with a deaths/cases and a country dropdown that displays the timeseries plot
3. **CovidPieChart**: a component with a deaths/cases and a country dropdown that displays the pie chart plot
4. **CovidComponent**: a component that combines a `CovidTimeSeries` and a `CovidPieChart`, and adds its own dropdowns that control both subcomponents
5. **CovidDashboard**: a dashboard consisting of four `CovidComponent`s in four different tabs:
    1. One showing only european countries
    2. One showing only Asian countries
    3. One showing only cases
    4. One showing only deaths
    
    
So the `CovidComponent` gets re-used four times, with different configurations. And each `CovidComponent` reuses `CovidTimeSeries` and `CovidPieChart`. 

The `CovidPlots` figure factory gets passed down by the `CovidDashboard` to the `CovidComponents`, and by the `CovidComponents` to the `CovidTimeSeries` and `CovidPieChart`. 




# Running the dashboard

We run the dashboard by passing the `CovidDashboard` to a `DashApp` and calling `.run()`. 

We can store the configuration with `dashboard.to_yaml("covid_dashboard.yaml")`.

And then build a new `DashApp` instance directly from that configuration with `DashApp.from_yaml("covid_dashboard.yaml")

## Imports

In [ ]:
#hide

!pip install pandas plotly-express

You should consider upgrading via the '/Users/oege/projects/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
#hide
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc

from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import pandas as pd
import plotly.express as px

## CovidPlots

First we build a plot factory that holds the dataframe, and has two plotting functions:

- `plot_time_series`: plots a time series for a given list of countries for a given metric ('cases' or 'deaths')
- `plot_pie_chart`: plots a pie chart for a given list of countries for a given metric ('cases' or 'deaths')
    
We store a list of countries and metrics to `self.countries` and `self.metric`, so that the components can use those to populate the dropdowns. 

In [ ]:
class CovidPlots(DashFigureFactory):
    def __init__(self, datafile="covid.csv", exclude_countries=[]):
        super().__init__()
        self.df = pd.read_csv(datafile)
        if exclude_countries:
            self.df = self.df[~self.df.countriesAndTerritories.isin(exclude_countries)]
        self.countries = self.df.countriesAndTerritories.unique().tolist()
        self.metrics = ['cases', 'deaths']
        
    def plot_time_series(self, countries, metric):
        return px.line(
            data_frame=self.df[self.df.countriesAndTerritories.isin(countries)],
            x='dateRep',
            y=metric,
            color='countriesAndTerritories',
            labels={'countriesAndTerritories':'Countries', 'dateRep':'date'},
            )
    
    def plot_pie_chart(self, countries, metric):
        return px.pie(
            data_frame=self.df[self.df.countriesAndTerritories.isin(countries)],
            names='countriesAndTerritories',
            values=metric,
            hole=.3,
            labels={'countriesAndTerritories':'Countries'}
            ) 
    
plot_factory = CovidPlots(datafile="covid.csv")
print(plot_factory.to_yaml())

dash_figure_factory:
  name: CovidPlots
  module: __main__
  params:
    datafile: covid.csv
    exclude_countries: []



## CovidTimeSeries

A `CovidTimeSeries` component consisting of:

    - A dropdown to select a number of countries
    - A dropdown to select cases or deaths
    - a time series plot
    
All parameters are automatically assigned to attributes. So we can simply use `self.plot_factory` or `self.countries` with having to specify the assignments in the `__init__`. 

Both dropdowns can be hidden by passing `hide_country_dropdown=True` or `hide_metric_dropdown=True`. This works
by using the `make_hideable()` staticmethod from the `DashComponent` class: We wrap the relevant part of the layout in a `self.make_hideable(..., hide=self.hide_country_dropdown)` wrapper. If `hide==True` then `make_hideable()` will wrap a hidden div around the element.

If no `include_countries` or `include_metrics` are given, then by default all countries and metrics get included.

In [ ]:
class CovidTimeSeries(DashComponent):
    def __init__(self, plot_factory, 
                 hide_country_dropdown=False, include_countries=None, countries=None, 
                 hide_metric_dropdown=False, include_metrics=None, metric='cases'):
        super().__init__()
        
        if not self.include_countries:
            self.include_countries = self.plot_factory.countries
        if not self.countries:
            self.countries = self.include_countries
        
        if not self.include_metrics:
            self.include_metrics = self.plot_factory.metrics
        if not self.metric:
            self.metric = self.include_metrics[0]
        
    def layout(self):
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H3("Covid Time Series"),
                    self.make_hideable(
                        dcc.Dropdown(
                            id='timeseries-metric-dropdown-'+self.name,
                            options=[{'label': metric, 'value': metric} for metric in self.include_metrics],
                            value=self.metric,
                        ), hide=self.hide_metric_dropdown),
                    self.make_hideable(
                        dcc.Dropdown(
                            id='timeseries-country-dropdown-'+self.name,
                            options=[{'label': country, 'value': country} for country in self.include_countries],
                            value=self.countries,
                            multi=True,
                        ), hide=self.hide_country_dropdown),
                    dcc.Graph(id='timeseries-figure-'+self.name)
                ]),
            ])
        ])
    
    def _register_callbacks(self, app):
        @app.callback(
            Output('timeseries-figure-'+self.name, 'figure'),
            Input('timeseries-country-dropdown-'+self.name, 'value'),
            Input('timeseries-metric-dropdown-'+self.name, 'value')
        )
        def update_timeseries_plot(countries, metric):
            if countries and metric:
                return self.plot_factory.plot_time_series(countries, metric)
            raise PreventUpdate

## CovidPieChart

A `CovidPieChart` component consisting of:

    - A dropdown to select a number of countries
    - A dropdown to select cases or deaths
    - a pie chart plot
    
The first parameter is a `CovidPlots` `DashPlotFactory` that will be used to return the right plot in the callbacks.
    
All parameters are automatically assigned to attributes. So we can simply use `self.plot_factory` or `self.countries` with having to specify the assignments in the `__init__`. 

Both dropdowns can be hidden by passing `hide_country_dropdown=True` or `hide_metric_dropdown=True`. This works
by using the `make_hideable()` staticmethod from the `DashComponent` class: We wrap the relevant part of the layout in a `self.make_hideable(..., hide=self.hide_country_dropdown)` wrapper. If `hide==True` then `make_hideable()` will wrap a hidden div around the element.

If no `include_countries` or `include_metrics` are given, then by default all countries and metrics get included.

In [ ]:
class CovidPieChart(DashComponent):
    def __init__(self, plot_factory, 
                 hide_country_dropdown=False, include_countries=None, countries=None, 
                 hide_metric_dropdown=False, include_metrics=None, metric='cases'):
        super().__init__()
        
        if not self.include_countries:
            self.include_countries = self.plot_factory.countries
        if not self.countries:
            self.countries = self.include_countries
        
        if not self.include_metrics:
            self.include_metrics = self.plot_factory.metrics
        if not self.metric:
            self.metric = self.include_metrics[0]
        
    def layout(self):
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H3("Covid Pie Chart"),
                    self.make_hideable(
                        dcc.Dropdown(
                            id='piechart-metric-dropdown-'+self.name,
                            options=[{'label': metric, 'value': metric} for metric in self.include_metrics],
                            value=self.metric,
                        ), hide=self.hide_metric_dropdown),
                    self.make_hideable(
                        dcc.Dropdown(
                            id='piechart-country-dropdown-'+self.name,
                            options=[{'label': country, 'value': country} for country in self.include_countries],
                            value=self.countries,
                            multi=True
                        ), hide=self.hide_country_dropdown),
                    dcc.Graph(id='piechart-figure-'+self.name)
                ]),
            ])
        ])
    
    def _register_callbacks(self, app):
        @app.callback(
            Output('piechart-figure-'+self.name, 'figure'),
            Input('piechart-country-dropdown-'+self.name, 'value'),
            Input('piechart-metric-dropdown-'+self.name, 'value')
        )
        def update_timeseries_plot(countries, metric):
            if countries and metric:
                return self.plot_factory.plot_pie_chart(countries, metric)
            raise PreventUpdate

## CovidComposite

A `CovidComposite` combines a `CovidTimeSeries` and `CovidPieChart` component into a single composite component.

- The `plot_factory` gets passed down to the subcomponents
- The subcomponents are included in the layout with `self.timeseries.layout()` and `self.piechart.layout()`.
- An additional callback is added with `def _register_components(self, app)` (**note the underscore!**) that connects the dropdowns of the `CovidComponent` to the dropdowns of the subcomponents. (therefore the subcomponent dropdowns are also hidden by setting `hide_country_dropdown=True` and `hide_metric_dropdown=True`)


In [ ]:
class CovidComponent(DashComponent):
    def __init__(self, plot_factory, title="Covid Analysis",
                 hide_country_dropdown=False, 
                 include_countries=None, countries=None, 
                 hide_metric_dropdown=False, 
                 include_metrics=None, metric='cases'):
        super().__init__(title=title)
        
        if not self.include_countries:
            self.include_countries = self.plot_factory.countries
        if not self.countries:
            self.countries = self.include_countries
        
        if not self.include_metrics:
            self.include_metrics = self.plot_factory.metrics
        if not self.metric:
            self.metric = self.include_metrics[0]
            
        self.timeseries = CovidTimeSeries(
                plot_factory, 
                hide_country_dropdown=True, countries=self.countries,
                hide_metric_dropdown=True, metric=self.metric)
        
        self.piechart = CovidPieChart(
                plot_factory, 
                hide_country_dropdown=True, countries=self.countries,
                hide_metric_dropdown=True, metric=self.metric)
        
    def layout(self):
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H1(self.title),
                    self.make_hideable(
                        dcc.Dropdown(
                            id='dashboard-metric-dropdown-'+self.name,
                            options=[{'label': metric, 'value': metric} for metric in self.include_metrics],
                            value=self.metric,
                        ), hide=self.hide_metric_dropdown),
                    self.make_hideable(
                        dcc.Dropdown(
                            id='dashboard-country-dropdown-'+self.name,
                            options=[{'label': metric, 'value': metric} for metric in self.include_countries],
                            value=self.countries,
                            multi=True,
                        ), hide=self.hide_country_dropdown),
                ], md=6),
            ], justify="center"),
            dbc.Row([
                dbc.Col([
                    self.timeseries.layout(),
                ], md=6),
                dbc.Col([
                    self.piechart.layout(),
                ], md=6)
            ])
        ], fluid=True)
    
    def _register_callbacks(self, app):
        @app.callback(
            Output('timeseries-country-dropdown-'+self.timeseries.name, 'value'),
            Output('piechart-country-dropdown-'+self.piechart.name, 'value'),
            Input('dashboard-country-dropdown-'+self.name, 'value'),
        )
        def update_timeseries_plot(countries):
            return countries, countries
        
        @app.callback(
            Output('timeseries-metric-dropdown-'+self.timeseries.name, 'value'),
            Output('piechart-metric-dropdown-'+self.piechart.name, 'value'),
            Input('dashboard-metric-dropdown-'+self.name, 'value'),
        )
        def update_timeseries_plot(metric):
            return metric, metric
        

# CovidDashboard

The final `CovidDashboard` consists of four `CovidComposites` subcomponents, each in its own tab, and each with a slightly different configuration.

- First tab only shows European counties, by passing `include_countries=[...]`
- Second tab only shows Asian countries
- Third includes all countries, but only shows cases, and hides the metric dropdown. 
    The default initial countries are ['China', 'Italy', 'Brazil']
- Third includes all countries, but only shows deaths, and hides the metric dropdown. 
    The default initial countries are ['China', 'Italy', 'Brazil']


In [ ]:
class CovidDashboard(DashComponent):
    def __init__(self, plot_factory, 
                 europe_countries = ['Italy',  'Spain', 'Germany', 'France', 'Iran', 
                                     'United_Kingdom', 'Switzerland', 'Netherlands',  
                                     'Belgium', 'Austria', 'Portugal', 'Norway'],
                asia_countries = ['China', 'Vietnam', 'Malaysia', 'Philippines', 
                                  'Taiwan', 'Myanmar', 'Thailand', 'South_Korea', 'Japan']):
        super().__init__()
        
        self.europe = CovidComponent(self.plot_factory, "Europe", 
                                     include_countries=self.europe_countries)
        self.asia = CovidComponent(self.plot_factory, "Asia", include_countries=self.asia_countries)
        self.cases_only = CovidComponent(self.plot_factory, "Cases Only", 
                                         include_metrics=['cases'], metric='cases',
                                         hide_metric_dropdown=True,
                                         countries=['China', 'Italy', 'Brazil'])
        self.deaths_only = CovidComponent(self.plot_factory, "Deaths Only", 
                                          include_metrics=['deaths'], metric='deaths',
                                          hide_metric_dropdown=True,
                                          countries=['China', 'Italy', 'Brazil'])
        
    def layout(self):
        return dbc.Container([
            dbc.Row([
                html.H1("Covid Dashboard"),
            ]),
            dbc.Row([
                dbc.Col([
                    dcc.Tabs(id="tabs", value=self.europe.name, 
                        children=[
                            dcc.Tab(label=self.europe.title, 
                                    id=self.europe.name, 
                                    value=self.europe.name,
                                    children=self.europe.layout()),
                            dcc.Tab(label=self.asia.title, 
                                    id=self.asia.name, 
                                    value=self.asia.name,
                                    children=self.asia.layout()),
                            dcc.Tab(label=self.cases_only.title, 
                                    id=self.cases_only.name, 
                                    value=self.cases_only.name,
                                    children=self.cases_only.layout()),
                            dcc.Tab(label=self.deaths_only.title, 
                                    id=self.deaths_only.name, 
                                    value=self.deaths_only.name,
                                    children=self.deaths_only.layout()),
                        ]),
                ])
            ])
        ], fluid=True)
    
dashboard = CovidDashboard(plot_factory)
print(dashboard.to_yaml())

dash_component:
  name: CovidDashboard
  module: __main__
  params:
    plot_factory:
      dash_figure_factory:
        name: CovidPlots
        module: __main__
        params:
          datafile: covid.csv
          exclude_countries: []
    europe_countries:
    - Italy
    - Spain
    - Germany
    - France
    - Iran
    - United_Kingdom
    - Switzerland
    - Netherlands
    - Belgium
    - Austria
    - Portugal
    - Norway
    asia_countries:
    - China
    - Vietnam
    - Malaysia
    - Philippines
    - Taiwan
    - Myanmar
    - Thailand
    - South_Korea
    - Japan



## Start app

Pass the `dashboard` to the `DashApp`, and add the bootstrap stylesheet that is needed to correctly display all the `dbc.Row`s and `dbc.Col`s:

In [ ]:
app = DashApp(dashboard_component, external_stylesheets=[dbc.themes.BOOTSTRAP])

And finally we run the app:


In [ ]:
if not True: # remove to run
    app.run()

## Store App config and reload

We can check out the configuration that was generated for our dashboard. 
This includes:

- What the topline dashboard_component is, from where to import it and with which parameters to start it
- The `plot_factory` parameter gets automatically replaced with the configuration for the plot_factory:
    - includes information on how to import and all parameters
- The default list of parameters for `europe_countries` and `asia_countries` is included
- The parameters for the `dash_app` itself:
    - port=8050
    - mode='dash'
    - external_stylesheets=dbc.themes.BOOTSTRAP

In [ ]:
print(app.to_yaml())

dash_app:
  name: DashApp
  module: dash_oop_components.core
  params:
    dashboard_component:
      dash_component:
        name: CovidDashboard
        module: __main__
        params:
          plot_factory:
            dash_figure_factory:
              name: CovidPlots
              module: __main__
              params:
                datafile: covid.csv
                include_countries: []
          europe_countries:
          - Italy
          - Spain
          - Germany
          - France
          - Iran
          - United_Kingdom
          - Switzerland
          - Netherlands
          - Belgium
          - Austria
          - Portugal
          - Norway
          asia_countries:
          - China
          - Vietnam
          - Malaysia
          - Philippines
          - Taiwan
          - Myanmar
          - Thailand
          - South_Korea
          - Japan
    port: 8050
    mode: dash
    kwargs:
      external_stylesheets:
      - https://stackpath.bootstrapcdn.co

Now we can store this configuration to disk:

In [ ]:
app.to_yaml("covid_dashboard.yaml")

And reload another `DashApp` from this configuration. This means that:

- The releve3nt `DashFigureFactory`s will get imported, instantiated with the right parameters and passed down to the `DashComponent`.
- The `DashComponent`s will get imported from the right module, passed an instantiated `DashFigureFactory` along with other parameters.
- Either a `dash.Dash()` or a `jupyter_dash.JupyterDash()` app gets started, depending on `mode`.

In [ ]:
app2 = DashApp.from_yaml("covid_dashboard.yaml")

We can check that the configuration of this new `app2` is indeed the same as `app`:

In [ ]:
print(app2.to_yaml())

dash_app:
  name: DashApp
  module: dash_oop_components.core
  params:
    dashboard_component:
      dash_component:
        name: CovidDashboard
        module: __main__
        params:
          plot_factory:
            dash_figure_factory:
              name: CovidPlots
              module: __main__
              params:
                datafile: covid.csv
                include_countries: []
          europe_countries:
          - Italy
          - Spain
          - Germany
          - France
          - Iran
          - United_Kingdom
          - Switzerland
          - Netherlands
          - Belgium
          - Austria
          - Portugal
          - Norway
          asia_countries:
          - China
          - Vietnam
          - Malaysia
          - Philippines
          - Taiwan
          - Myanmar
          - Thailand
          - South_Korea
          - Japan
    port: 8050
    mode: dash
    kwargs:
      external_stylesheets:
      - https://stackpath.bootstrapcdn.co

And if we run it it still works!


In [ ]:
if not True: # remove to run
    app2.run()